# Lesson B6: Slices & String Handling

**Duration**: 75-90 minutes  
**Stage**: Beginner (Foundations)

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
1. Understand and use string slices (`&str`) effectively in Rust programs
2. Work with array slices to access portions of arrays and vectors
3. Handle UTF-8 text processing safely and correctly
4. Apply slicing techniques to solve real-world text processing problems
5. Distinguish between `String` and `&str` and choose the appropriate type

---

## 📋 Prerequisite Review

**Quick Check**: From our previous lessons:

1. What are the three ownership rules in Rust?
2. What's the difference between `&T` and `&mut T`?
3. How do you create a reference to a variable?
4. What happens when you try to use a value after it's been moved?

**Answers**: 1) Each value has one owner, ownership can be transferred, owner is responsible for cleanup, 2) Immutable vs mutable references, 3) Use `&` operator, 4) Compile error - value is no longer accessible

---

## 🧠 Key Concepts

### What are Slices?

**Slices** are references to a contiguous sequence of elements in a collection:
- **No ownership**: Slices borrow data, they don't own it
- **Dynamic size**: The size is determined at runtime
- **Safe access**: Bounds checking prevents buffer overflows
- **Efficient**: Zero-cost abstraction over raw pointers

### String Types in Rust

- **`String`**: Owned, growable, heap-allocated string
- **`&str`**: String slice, borrowed reference to string data
- **String literals**: `"hello"` are `&str` with `'static` lifetime
- **UTF-8**: All Rust strings are valid UTF-8

### Slice Syntax

- **Full slice**: `&array[..]`
- **Range slice**: `&array[start..end]` (end is exclusive)
- **From start**: `&array[start..]`
- **To end**: `&array[..end]`

---

## 🔬 Live Code Exploration

### String Slices Fundamentals

In [ ]:
// String slice basics
fn string_slice_demo() {
    println!("=== String Slice Fundamentals ===");
    
    // String literal is a &str
    let greeting: &str = "Hello, Rust!";
    println!("String literal: {}", greeting);
    println!("Length: {} characters", greeting.len());
    
    // Creating a String and taking slices
    let owned_string = String::from("Rust Programming Language");
    
    // Different ways to slice
    let full_slice = &owned_string[..];
    let first_word = &owned_string[0..4];   // "Rust"
    let from_fifth = &owned_string[5..];    // "Programming Language"
    let last_word = &owned_string[17..];    // "Language"
    
    println!("\nSlicing examples:");
    println!("Full slice: '{}'", full_slice);
    println!("First word: '{}'", first_word);
    println!("From fifth: '{}'", from_fifth);
    println!("Last word: '{}'", last_word);
    
    // Safe slicing with get method
    match owned_string.get(0..4) {
        Some(slice) => println!("Safe slice: '{}'", slice),
        None => println!("Invalid slice range"),
    }
    
    // This would panic: &owned_string[0..100]
    // But this is safe:
    match owned_string.get(0..100) {
        Some(slice) => println!("Large slice: '{}'", slice),
        None => println!("Slice range too large - safely handled!"),
    }
}

string_slice_demo();

### Array and Vector Slices

In [ ]:
// Array and vector slicing
fn array_slice_demo() {
    println!("\n=== Array and Vector Slices ===");
    
    // Array slicing
    let numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10];
    
    let first_half = &numbers[0..5];
    let second_half = &numbers[5..];
    let middle = &numbers[3..7];
    
    println!("Original array: {:?}", numbers);
    println!("First half: {:?}", first_half);
    println!("Second half: {:?}", second_half);
    println!("Middle section: {:?}", middle);
    
    // Vector slicing
    let mut vec = vec!["apple", "banana", "cherry", "date", "elderberry"];
    
    let fruits_slice = &vec[1..4];
    println!("\nFruits slice: {:?}", fruits_slice);
    
    // Slices can be passed to functions
    print_slice(&vec[..3]);
    print_slice(&numbers[5..8]);
    
    // Mutable slices
    let mut_slice = &mut vec[1..3];
    mut_slice[0] = "blueberry";
    println!("After modification: {:?}", vec);
}

fn print_slice<T: std::fmt::Debug>(slice: &[T]) {
    println!("Slice contents: {:?}", slice);
}

array_slice_demo();

### UTF-8 and Text Processing

In [ ]:
// UTF-8 text processing
fn utf8_demo() {
    println!("\n=== UTF-8 Text Processing ===");
    
    // UTF-8 strings with international characters
    let international = "Hello 世界 🦀 Rust!";
    println!("International text: {}", international);
    println!("Byte length: {}", international.len());
    println!("Character count: {}", international.chars().count());
    
    // Iterating over characters vs bytes
    println!("\nCharacter iteration:");
    for (i, ch) in international.chars().enumerate() {
        println!("  {}: '{}'", i, ch);
    }
    
    println!("\nByte iteration (first 20 bytes):");
    for (i, byte) in international.bytes().take(20).enumerate() {
        println!("  {}: {} (0x{:02x})", i, byte, byte);
    }
    
    // Safe character-based slicing
    let text = "Programming";
    
    // This works because all characters are ASCII (1 byte each)
    let prog = &text[0..4];
    println!("\nASCII slice: '{}'", prog);
    
    // For Unicode text, use char indices
    let unicode_text = "Prüfung";
    let char_indices: Vec<_> = unicode_text.char_indices().collect();
    println!("\nCharacter indices in '{}': {:?}", unicode_text, char_indices);
    
    // Safe Unicode slicing
    if let Some((start, _)) = char_indices.get(0) {
        if let Some((end, _)) = char_indices.get(3) {
            let safe_slice = &unicode_text[*start..*end];
            println!("Safe Unicode slice: '{}'", safe_slice);
        }
    }
}

utf8_demo();

### Practical String Processing

In [ ]:
// Practical string processing functions
fn first_word(s: &str) -> &str {
    let bytes = s.as_bytes();
    
    for (i, &item) in bytes.iter().enumerate() {
        if item == b' ' {
            return &s[0..i];
        }
    }
    
    &s[..] // Return entire string if no space found
}

fn last_word(s: &str) -> &str {
    match s.rfind(' ') {
        Some(pos) => &s[pos + 1..],
        None => s,
    }
}

fn word_count(s: &str) -> usize {
    s.split_whitespace().count()
}

fn extract_domain(email: &str) -> Option<&str> {
    email.find('@').map(|pos| &email[pos + 1..])
}

fn truncate_with_ellipsis(s: &str, max_len: usize) -> String {
    if s.len() <= max_len {
        s.to_string()
    } else {
        format!("{}...", &s[0..max_len.saturating_sub(3)])
    }
}

fn practical_string_processing() {
    println!("\n=== Practical String Processing ===");
    
    let sentence = "The quick brown fox jumps over the lazy dog";
    
    println!("Original: '{}'", sentence);
    println!("First word: '{}'", first_word(sentence));
    println!("Last word: '{}'", last_word(sentence));
    println!("Word count: {}", word_count(sentence));
    
    // Email processing
    let emails = ["user@example.com", "admin@rust-lang.org", "invalid-email"];
    
    println!("\nEmail domain extraction:");
    for email in &emails {
        match extract_domain(email) {
            Some(domain) => println!("  {} -> domain: {}", email, domain),
            None => println!("  {} -> invalid email format", email),
        }
    }
    
    // Text truncation
    let long_text = "This is a very long piece of text that needs to be truncated";
    println!("\nText truncation:");
    println!("Original ({}): {}", long_text.len(), long_text);
    println!("Truncated to 20: {}", truncate_with_ellipsis(long_text, 20));
    println!("Truncated to 10: {}", truncate_with_ellipsis(long_text, 10));
    
    // Working with lines
    let multiline = "Line 1\nLine 2\nLine 3\nLine 4";
    println!("\nLine processing:");
    for (i, line) in multiline.lines().enumerate() {
        println!("  {}: '{}'", i + 1, line);
    }
    
    // String splitting and joining
    let csv_data = "apple,banana,cherry,date";
    let fruits: Vec<&str> = csv_data.split(',').collect();
    println!("\nCSV parsing: {:?}", fruits);
    
    let rejoined = fruits.join(" | ");
    println!("Rejoined: {}", rejoined);
}

practical_string_processing();

---

## 🎯 Guided Practice

### Exercise 1: Text Analysis Tool

Create functions to analyze text using slices.

In [ ]:
// TODO: Complete the text analysis functions

fn count_vowels(text: &str) -> usize {
    // TODO: Count vowels (a, e, i, o, u) in the text
    text.chars()
        .filter(|c| "aeiouAEIOU".contains(*c))
        .count()
}

fn find_longest_word(text: &str) -> &str {
    // TODO: Find the longest word in the text
    text.split_whitespace()
        .max_by_key(|word| word.len())
        .unwrap_or("")
}

fn reverse_words(text: &str) -> String {
    // TODO: Reverse the order of words in the text
    text.split_whitespace()
        .rev()
        .collect::<Vec<_>>()
        .join(" ")
}

fn extract_initials(name: &str) -> String {
    // TODO: Extract initials from a full name
    name.split_whitespace()
        .filter_map(|word| word.chars().next())
        .collect::<String>()
        .to_uppercase()
}

fn is_palindrome(text: &str) -> bool {
    // TODO: Check if text is a palindrome (ignoring spaces and case)
    let cleaned: String = text.chars()
        .filter(|c| !c.is_whitespace())
        .map(|c| c.to_lowercase().next().unwrap())
        .collect();
    
    cleaned == cleaned.chars().rev().collect::<String>()
}

fn text_analysis_demo() {
    println!("\n=== Text Analysis Demo ===");
    
    let sample_text = "The quick brown fox jumps over the lazy dog";
    
    println!("Text: '{}'", sample_text);
    println!("Vowel count: {}", count_vowels(sample_text));
    println!("Longest word: '{}'", find_longest_word(sample_text));
    println!("Reversed words: '{}'", reverse_words(sample_text));
    
    let names = ["John Doe", "Mary Jane Watson", "Peter Parker"];
    println!("\nInitials extraction:");
    for name in &names {
        println!("  {} -> {}", name, extract_initials(name));
    }
    
    let test_phrases = ["racecar", "A man a plan a canal Panama", "hello world"];
    println!("\nPalindrome check:");
    for phrase in &test_phrases {
        println!("  '{}' -> {}", phrase, is_palindrome(phrase));
    }
}

text_analysis_demo();

---

## 🚀 Independent Practice

### Challenge: Log File Parser

Create a log file parser using string slices.

In [ ]:
// TODO: Implement a log file parser

#[derive(Debug)]
struct LogEntry<'a> {
    timestamp: &'a str,
    level: &'a str,
    message: &'a str,
}

fn parse_log_line(line: &str) -> Option<LogEntry> {
    // TODO: Parse log line format: "[timestamp] LEVEL: message"
    // Example: "[2024-01-15 10:30:45] INFO: Server started successfully"
    
    if !line.starts_with('[') {
        return None;
    }
    
    let close_bracket = line.find(']')?;
    let timestamp = &line[1..close_bracket];
    
    let rest = &line[close_bracket + 1..].trim_start();
    let colon_pos = rest.find(':')?;
    
    let level = rest[..colon_pos].trim();
    let message = rest[colon_pos + 1..].trim();
    
    Some(LogEntry {
        timestamp,
        level,
        message,
    })
}

fn filter_by_level<'a>(entries: &[LogEntry<'a>], level: &str) -> Vec<&LogEntry<'a>> {
    // TODO: Filter log entries by level
    entries.iter()
        .filter(|entry| entry.level.eq_ignore_ascii_case(level))
        .collect()
}

fn extract_error_messages<'a>(entries: &[LogEntry<'a>]) -> Vec<&'a str> {
    // TODO: Extract just the messages from ERROR level entries
    entries.iter()
        .filter(|entry| entry.level.eq_ignore_ascii_case("ERROR"))
        .map(|entry| entry.message)
        .collect()
}

fn count_by_level(entries: &[LogEntry]) -> std::collections::HashMap<&str, usize> {
    // TODO: Count entries by log level
    let mut counts = std::collections::HashMap::new();
    for entry in entries {
        *counts.entry(entry.level).or_insert(0) += 1;
    }
    counts
}

fn log_parser_demo() {
    println!("\n=== Log File Parser Demo ===");
    
    let log_data = r#"[2024-01-15 10:30:45] INFO: Server started successfully
[2024-01-15 10:30:46] DEBUG: Loading configuration file
[2024-01-15 10:30:47] INFO: Database connection established
[2024-01-15 10:30:50] WARN: High memory usage detected
[2024-01-15 10:30:55] ERROR: Failed to connect to external API
[2024-01-15 10:31:00] INFO: Retrying API connection
[2024-01-15 10:31:05] ERROR: API connection failed again
[2024-01-15 10:31:10] INFO: Falling back to cached data"#;
    
    // Parse all log entries
    let entries: Vec<LogEntry> = log_data.lines()
        .filter_map(parse_log_line)
        .collect();
    
    println!("Parsed {} log entries:", entries.len());
    for entry in &entries {
        println!("  {} [{}]: {}", entry.timestamp, entry.level, entry.message);
    }
    
    // Filter by level
    let errors = filter_by_level(&entries, "ERROR");
    println!("\nERROR entries ({}):", errors.len());
    for error in &errors {
        println!("  {}: {}", error.timestamp, error.message);
    }
    
    // Extract error messages
    let error_messages = extract_error_messages(&entries);
    println!("\nError messages:");
    for msg in &error_messages {
        println!("  - {}", msg);
    }
    
    // Count by level
    let counts = count_by_level(&entries);
    println!("\nLog level counts:");
    for (level, count) in &counts {
        println!("  {}: {}", level, count);
    }
}

log_parser_demo();

---

## 🧪 Active Recall Checkpoint

**Test your understanding without looking back:**

1. What's the difference between `String` and `&str`?
2. How do you create a slice of an array from index 2 to 5?
3. Why might `&text[0..10]` panic but `text.get(0..10)` won't?
4. What does `text.chars().count()` return vs `text.len()`?
5. How do you safely slice a Unicode string?
6. What happens to the original data when you create a slice?
7. Can you modify data through an immutable slice?
8. What's the lifetime relationship between a slice and its source data?

**Write your answers below:**

**Your Answers:**
1. 
2. 
3. 
4. 
5. 
6. 
7. 
8. 

---

## 🤔 Reflection Prompt

Consider these questions:

1. **How do slices help with memory efficiency in Rust programs?**
2. **When would you choose `&str` over `String` and vice versa?**
3. **How does Rust's UTF-8 handling compare to other languages you know?**
4. **What are the safety benefits of Rust's slice bounds checking?**

Write your thoughts below:

**Your Reflections:**

1. 

2. 

3. 

4. 

---

## 🔮 Preview & Connections

### Coming Up Next: Compound Data Types

In our next lesson, you'll learn about:
- Tuples for grouping different types
- Arrays for fixed-size collections
- Vectors for dynamic collections
- When to use each data structure

### How This Connects
String slices are fundamental to understanding:
- How Rust handles text efficiently
- The relationship between ownership and borrowing
- Memory-safe programming without garbage collection
- Building blocks for more complex data structures

---

## ✅ Expected Outcomes

**Self-Assessment Checklist** - Can you:

- [ ] Create and use string slices (`&str`) effectively?
- [ ] Slice arrays and vectors safely?
- [ ] Handle UTF-8 text processing correctly?
- [ ] Choose between `String` and `&str` appropriately?
- [ ] Use slice syntax for different ranges?
- [ ] Process text data using slice-based functions?
- [ ] Understand the memory implications of slices?

If you checked all boxes, excellent! You've mastered Rust's slice system.

---

**🎉 Great Progress!** You now understand how to work efficiently with text and array data in Rust!